# Model training

In this notebook, we will train our first model and apply this model to a soundscape. We will keep the amount of training samples, species and soundscapes to a minimum to keep the execution time as short as possible. Remember, this is only a sample implementation, feel free to explore your own workflow.

These are the steps that we will cover:


* select audio files we want to use for training  
* extract spectrograms from those files and save them in a working directory  
* load selected samples into a large in-memory dataset  
* build a simple beginners CNN  
* train the model  
* apply the model to a selected soundscape and look at the results 

# 1. Settings and imports

Let’s begin with imports and a few basic settings.

In [ ]:
import os

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import librosa
import numpy as np

from sklearn.utils import shuffle
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf

# !pip install imutils
# from imutils import paths

# Global vars
RANDOM_SEED = 1337
SAMPLE_RATE = 32000
SIGNAL_LENGTH = 5.0 # 5 seconds
SPEC_LENGTH = 5.0 # 5 seconds
SPEC_SHAPE = (48, 128) # height x width
# SPEC_SHAPE = (257, 384) # height x width
FMIN = 500
FMAX = 12500
MAX_AUDIO_FILES = 3000

# 2. Data preparation

The training data for this competition contains tens of thousands of audio files for 397 species. That’s way too much for this tutorial, so we will limit our species selection to species that have at least 200 recordings with a rating of 4 or better.

In [ ]:
# Code adapted from: 
# https://www.kaggle.com/frlemarchand/bird-song-classification-using-an-efficientnet
# Make sure to check out the entire notebook.

# Load metadata file
train = pd.read_csv('../input/birdclef-2021/train_metadata.csv',)

# Limit the number of training samples and classes
# First, only use high quality samples
train = train.query('rating>=4')

# Second, assume that birds with the most training samples are also the most common
# A species needs at least 200 recordings with a rating above 4 to be considered common
birds_count = {}
for bird_species, count in zip(train.primary_label.unique(), 
                               train.groupby('primary_label')['primary_label'].count().values):
    birds_count[bird_species] = count
most_represented_birds = [key for key,value in birds_count.items() if value >= 300] 

TRAIN = train.query('primary_label in @most_represented_birds')
LABELS = sorted(TRAIN.primary_label.unique())

# Let's see how many species and samples we have left
print('NUMBER OF SPECIES IN TRAIN DATA:', len(LABELS))
print('NUMBER OF SAMPLES IN TRAIN DATA:', len(TRAIN))
print('LABELS:', most_represented_birds)

Ok, that leaves us with 27 species and 8,548 audio files. The species list includes very common species like the House Sparrow (houspa), Blue Jay (blujay), or Song Sparrow (sonspa). This is not a bad selection to start experimenting.

# 3. Extract training samples

We need to define a function that extracts spectrograms for a given audio file. This function needs to load a file with Librosa (we only use the first 15 seconds in this tutorial), extract mel spectrograms and save each spectrogram as PNG image in a working directory for later access.

In [ ]:
import soundfile as sf
# Shuffle the training data and limit the number of audio files to MAX_AUDIO_FILES
TRAIN = shuffle(TRAIN, random_state=RANDOM_SEED)[:MAX_AUDIO_FILES]

# Define a function that splits an audio file, 
# extracts spectrograms and saves them in a working directory
def get_spectrograms(filepath, primary_label, output_dir):
    
    # Open the file with librosa (limited to the first 15 seconds)
    sig, rate = librosa.load(filepath, sr=SAMPLE_RATE, offset=None, duration=15)
    
    # Split signal into five second chunks
    sig_splits = []
    for i in range(0, len(sig), int(SIGNAL_LENGTH * SAMPLE_RATE)):
        split = sig[i:i + int(SIGNAL_LENGTH * SAMPLE_RATE)]

        # End of signal?
        if len(split) < int(SIGNAL_LENGTH * SAMPLE_RATE):
            break
        
        sig_splits.append(split)
        
    # Extract mel spectrograms for each audio chunk
    s_cnt = 0
    saved_samples = []
    for chunk in sig_splits:
        
        hop_length = int(SIGNAL_LENGTH * SAMPLE_RATE / (SPEC_SHAPE[1] - 1))
        mel_spec = librosa.feature.melspectrogram(y=chunk, 
                                                  sr=SAMPLE_RATE, 
                                                  n_fft=1024, 
                                                  hop_length=hop_length, 
                                                  n_mels=SPEC_SHAPE[0], 
                                                  fmin=FMIN, 
                                                  fmax=FMAX)
    
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max) 
        
        # Normalize
        mel_spec -= mel_spec.min()
        mel_spec /= mel_spec.max()

        # Save as image file
        save_dir = os.path.join(output_dir, primary_label)
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        save_path = os.path.join(save_dir, filepath.rsplit(os.sep, 1)[-1].rsplit('.', 1)[0] + 
                                 '_' + str(s_cnt) + '.png')
        
#         # save chunck soundfile
#         sf.write(save_path, chunk, SAMPLE_RATE, format='ogg', subtype='vorbis')
        
        im = Image.fromarray(mel_spec * 255.0).convert("L")
        im.save(save_path)
        
        saved_samples.append(save_path)
        s_cnt += 1
        
    return saved_samples

print('FINAL NUMBER OF AUDIO FILES IN TRAINING DATA:', len(TRAIN))

Ok, we have 1,500 audio files that cover 27 species, let's extract spectrograms (this might take a while).

In [ ]:
# Parse audio files and extract training samples
input_dir = '../input/birdclef-2021/train_short_audio/'
output_dir = '../working/melspectrogram_dataset/'
samples = []
with tqdm(total=len(TRAIN)) as pbar:
    for idx, row in TRAIN.iterrows():
        pbar.update(1)
        
        if row.primary_label in most_represented_birds:
            audio_file_path = os.path.join(input_dir, row.primary_label, row.filename)
            samples += get_spectrograms(audio_file_path, row.primary_label, output_dir)
            
TRAIN_SPECS = shuffle(samples, random_state=RANDOM_SEED)
print('SUCCESSFULLY EXTRACTED {} SPECTROGRAMS'.format(len(TRAIN_SPECS)))

Alright, we got 4,157 training spectrograms. That's roughly 150 for each species which is not too bad.

Let's make sure the spectrograms look right and show the first 12.

In [ ]:
# Plot the first 12 spectrograms of TRAIN_SPECS
plt.figure(figsize=(15, 7))
for i in range(12):
    spec = Image.open(TRAIN_SPECS[i])
    plt.subplot(3, 4, i + 1)
    plt.title(TRAIN_SPECS[i].split(os.sep)[-1])
    plt.imshow(spec, origin='lower')

Nice! These are good samples. Notice how some of them only contain a fraction of a bird call? That's an issue we won't deal with in this tutorial. We will simply ignore the fact that samples might not contain any bird sounds.

# 4. Load training samples

For now, our spectrograms reside in a working directory. If we want to train a model, we have to load them into memory. Yet, with potentially hundreds of thousands of extracted spectrograms, an in-memory dataset is not a good idea. But for now, loading samples from disk and combining them into a large NumPy array is fine. It’s the easiest way to use these data for training with Keras.

In [ ]:
# training data preparation
from imutils import paths

TRAIN_SPECS = list(paths.list_images('../input/gamanet-data-preparation/melspectrogram_dataset/'))
print("Total spectogram: {}".format(len(TRAIN_SPECS)))

In [ ]:
# Parse all samples and add spectrograms into train data, primary_labels into label data
train_specs, train_labels = [], []
with tqdm(total=len(TRAIN_SPECS)) as pbar:
    for path in TRAIN_SPECS:
        pbar.update(1)

        # Open image
        spec = Image.open(path)

        # Convert to numpy array
        spec = np.array(spec, dtype='float32')
        
        # Normalize between 0.0 and 1.0
        # and exclude samples with nan 
        spec -= spec.min()
        spec /= spec.max()
        if not spec.max() == 1.0 or not spec.min() == 0.0:
            continue

        # Add channel axis to 2D array
        spec = np.expand_dims(spec, -1)

        # Add new dimension for batch size
        spec = np.expand_dims(spec, 0)
        
#         # Open the file with librosa (limited to the first 15 seconds)
#         spec, rate = librosa.load(path, sr=SAMPLE_RATE, offset=None)

        # Add to train data
        if len(train_specs) == 0:
            train_specs = spec
        else:
            train_specs = np.vstack((train_specs, spec))

        # Add to label data
        target = np.zeros((len(LABELS)), dtype='float32')
        bird = path.split(os.sep)[-2]
        target[LABELS.index(bird)] = 1.0
        if len(train_labels) == 0:
            train_labels = target
        else:
            train_labels = np.vstack((train_labels, target))

# 5. Build a simple model

Alright, our dataset is ready, now we need to define a model architecture. In this tutorial, we’ll use a very simplistic, AlexNet-like design with four convolutional layers and three dense layers. It might make sense to choose an off-the-shelve TF model that was pre-trained on audio data, but we would need to adjust the inputs (i.e., the resolution of our spectrograms) to fit the external model. So we keep it simple and build our own model.

In [ ]:
from tensorflow.keras import datasets, layers, models, losses, Model

# Resnet50 Model
base_model = tf.keras.applications.ResNet50V2(
    include_top=False, weights=None, input_shape=(48, 128, 3))

for layer in base_model.layers:
    layer.trainable = True

x = layers.Flatten()(base_model.output)
x = layers.Dense(27, activation='relu')(x)
predictions = layers.Dense(27, activation = 'softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

import tensorflow as tf
from tensorflow import keras as k
from tensorflow.keras import layers as l

!cp ../input/birdnet-master/custom_layers.py .
import custom_layers

RANDOM_SEED = 1337

# Model config
FILTERS = [8, 16, 32, 64, 128]
KERNEL_SIZES =  [(5, 5), (3, 3), (3, 3), (3, 3), (3, 3)]
BRANCH_KERNEL_SIZE = (4, 10)
RESNET_K = 2
RESNET_N = 3
ACTIVATION = 'relu'
INITIALIZER = 'he'
DROPOUT_RATE = 0.33
NUM_CLASSES = 27

# Input config
SAMPLE_RATE = 48000
SPEC_LENGTH = 3.0 # 3 seconds
SPEC_SHAPE = (257, 384) # (height, width)

# Initializers
initializers = {'glorot': tf.initializers.GlorotNormal(seed=RANDOM_SEED),
                'he': k.initializers.he_normal(seed=RANDOM_SEED),
                'random': k.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=RANDOM_SEED),
                'constant': k.initializers.Constant(value=0)
                }

# Activations
activations = {'relu': l.ReLU(max_value=None, negative_slope=0.0, threshold=0.0),
               'elu': l.ELU(alpha=1.0),
               'lrelu': l.LeakyReLU(alpha=0.3)
              }

def resBlock(net_in, filters, kernel_size, stride=1, preactivated=True, block_id=1, name=''):

    # Show input shape
    print('    ' + name + ' IN SHAPE:', net_in.shape, end=' ')

    # Pre-activation
    if block_id > 1:
        net_pre = l.BatchNormalization(axis=-1, name=name + '_BN_PA')(net_in)
        net_pre = activations[ACTIVATION](net_pre)
    else:
        net_pre = net_in    

    # Pre-activated shortcut?
    if preactivated:
        net_in = net_pre

    # Bottleneck Convolution
    if stride > 1:
        net_pre = l.Conv2D(filters=net_pre.shape[-1],
                            kernel_size=1,
                            padding='same',
                            strides=1,
                            data_format='channels_last',
                            kernel_initializer=initializers[INITIALIZER],
                            name=name + '_CONV_1')(net_pre)
        net_pre = l.BatchNormalization(axis=-1, name=name + '_BN_1')(net_pre)
        net_pre = activations[ACTIVATION](net_pre)
    
    # First Convolution     
    net = l.Conv2D(filters=net_pre.shape[-1],
                   kernel_size=kernel_size,
                   padding='same',
                   strides=1,
                   data_format='channels_last',
                   kernel_initializer=initializers[INITIALIZER],
                   name=name + '_CONV_2')(net_pre)
    net = l.BatchNormalization(axis=-1, name=name + '_BN_2')(net)
    net = activations[ACTIVATION](net)

    # Pooling layer
    if stride > 1:
        net = l.MaxPooling2D(pool_size=(stride, stride), 
                             data_format='channels_last', 
                             name=name + '_POOL_1')(net)

    # Dropout Layer
    net = l.Dropout(rate=DROPOUT_RATE, seed=RANDOM_SEED, name=name + '_DO_1')(net)        

    # Second Convolution (make 1x1 if downsample block)
    if stride > 1:
        k_size = (1, 1)
    else:
        k_size =  kernel_size
    net = l.Conv2D(filters=filters,
                   kernel_size=k_size,
                   padding='same',
                   strides=1,
                   activation=None,
                   data_format='channels_last',
                   kernel_initializer=initializers[INITIALIZER],
                   name=name + '_CONV_3')(net)
    
    # Shortcut Layer
    if not net.shape[1:] == net_in.shape[1:]:        

        # Average pooling
        shortcut = l.AveragePooling2D(pool_size=(stride, stride), 
                                      data_format='channels_last',  
                                      name=name + '_SC_POOL')(net_in)

        # Shortcut convolution
        shortcut = l.Conv2D(filters=filters,
                            kernel_size=1,
                            padding='same',
                            strides=1,
                            activation=None,
                            data_format='channels_last',
                            kernel_initializer=initializers[INITIALIZER],
                            name=name + '_SC_CONV')(shortcut)
        
    else:

        # Shortcut = input
        shortcut = net_in
    
    # Merge Layer
    out = l.add([net, shortcut], name=name + '_ADD')

    # Show output shape
    print('OUT SHAPE:', out.shape)

    return out

def classificationBranch(net, kernel_size):

    # Post Convolution
    branch = l.Conv2D(filters=int(FILTERS[-1] * RESNET_K),
                      kernel_size=kernel_size,
                      strides=1,
                      data_format='channels_last',
                      kernel_initializer=initializers[INITIALIZER],
                      name='BRANCH_CONV_1')(net)
    branch = l.BatchNormalization(axis=-1, name='BRANCH_BN_1')(branch)
    branch = activations[ACTIVATION](branch)

    print('    POST  CONV SHAPE:', branch.shape)

    # Dropout Layer
    branch = l.Dropout(rate=DROPOUT_RATE, seed=RANDOM_SEED, name='BRANCH_DO_1')(branch)   
    
    # Dense Convolution
    branch = l.Conv2D(filters=int(FILTERS[-1] * RESNET_K * 2),
                      kernel_size=1,
                      strides=1,
                      data_format='channels_last',
                      kernel_initializer=initializers[INITIALIZER],
                      name='BRANCH_CONV_2')(branch)
    branch = l.BatchNormalization(axis=-1, name='BRANCH_BN_2')(branch)
    branch = activations[ACTIVATION](branch)

    print('    DENSE CONV SHAPE:', branch.shape)
    
    # Dropout Layer
    branch = l.Dropout(rate=DROPOUT_RATE, seed=RANDOM_SEED, name='BRANCH_DO_2')(branch)     

    # Class Convolution
    branch = l.Conv2D(filters=NUM_CLASSES,
                      kernel_size=1,
                      activation=None,
                      data_format='channels_last',
                      kernel_initializer=initializers[INITIALIZER],
                      name='BRANCH_CONV_3_' + str(NUM_CLASSES))(branch)

    return branch

def buildModel():

    print('BUILDING BirdNET MODEL...')

    # Input layer
    print('  INPUT:')
    inputs = k.Input(shape=(SPEC_SHAPE[0], SPEC_SHAPE[1]+1, 1), name='INPUT')

#     # Spectrogram layer if input is raw signal
#     net = custom_layers.SimpleSpecLayer(sample_rate=SAMPLE_RATE,
#                                 spec_shape=SPEC_SHAPE,
#                                 frame_step=int(SAMPLE_RATE * SPEC_LENGTH / (SPEC_SHAPE[1] + 1)),
#                                 data_format='channels_last',
#                                 name='SIMPLESPEC')(inputs)

    print('    INPUT LAYER  IN SHAPE:', inputs.shape)                       
#     print('    INPUT LAYER OUT SHAPE:', net.shape)

    # Preprocessing convolution
    print('  PRE-PROCESSING STEM:')
    net = l.Conv2D(filters=int(FILTERS[0] * RESNET_K),
                   kernel_size=KERNEL_SIZES[0],
                   strides=(2, 1),
                   padding='same',
                   data_format='channels_last',
                   kernel_initializer=initializers[INITIALIZER],
                   name='CONV_0')(inputs)

    # Batch norm layer
    net = l.BatchNormalization(axis=-1, name='BNORM_0')(net)
    net = activations[ACTIVATION](net)
    print('    FIRST  CONV OUT SHAPE:', net.shape)

    # Max pooling
    net = l.MaxPooling2D(pool_size=(2, 2), data_format='channels_last', name='POOL_0')(net)
    print('    FIRST  POOL OUT SHAPE:', net.shape)

    # Residual Stacks
    for i in range(1, len(FILTERS)):
        print('  RES STACK', i, ':')
        net = resBlock(net,
                       filters=int(FILTERS[i] * RESNET_K),
                       kernel_size=KERNEL_SIZES[i],
                       stride=2,
                       preactivated=True,
                       block_id=i,
                       name='BLOCK_' + str(i) + '-1')
        
        for j in range(1, RESNET_N):
            net = resBlock(net,
                           filters=int(FILTERS[i] * RESNET_K),
                           kernel_size=KERNEL_SIZES[i],
                           preactivated=False,
                           block_id=i+j,
                           name='BLOCK_' + str(i) + '-' + str(j + 1))

    # Post Activation
    net = l.BatchNormalization(axis=-1, name='BNORM_POST')(net)  
    net = activations[ACTIVATION](net)  

    # Classification branch
    print('  CLASS BRANCH:')
    net = classificationBranch(net,  BRANCH_KERNEL_SIZE) 
    print('    BRANCH OUT SHAPE:', net.shape)

    # Pooling
    net = l.GlobalAveragePooling2D(data_format='channels_last', name='GLOBAL_AVG_POOL')(net)
    print('  GLOBAL POOLING SHAPE:', net.shape)

    # Classification layer
    outputs = k.activations.sigmoid(net)
    print('  FINAL NET OUT SHAPE:', outputs.shape)

    # Build Keras model
    model = k.Model(inputs=inputs, outputs=outputs, name='BirdNET')

    # Print model stats
    print('...DONE!')
    #log.l(model.summary()) 
    
    print('MODEL HAS', (len([layer for layer in model.layers if len(layer.get_weights()) > 0])), 'WEIGHTED LAYERS (', len(model.layers), 'TOTAL )')
    print('MODEL HAS', model.count_params(), 'PARAMS')

    return model

def saveModel(model, name):

    print('SAVING MODEL...', end='')
    model.save(os.path.join('model', name)) 
    print('DONE!')


model = buildModel()
saveModel(model, 'BirdNET_1000_RAW_model.h5')

In [ ]:
# Make sure your experiments are reproducible
tf.random.set_seed(RANDOM_SEED)

# Build a simple model as a sequence of  convolutional blocks.
# Each block has the sequence CONV --> RELU --> BNORM --> MAXPOOL.
# Finally, perform global average pooling and add 2 dense layers.
# The last layer is our classification layer and is softmax activated.
# (Well it's a multi-label task so sigmoid might actually be a better choice)
model = tf.keras.Sequential([
    
    # First conv block
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', 
                           input_shape=(SPEC_SHAPE[0], SPEC_SHAPE[1], 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Second conv block
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)), 
    
    # Third conv block
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)), 
    
    # Fourth conv block
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Global pooling instead of flatten()
    tf.keras.layers.GlobalAveragePooling2D(), 
    
    # Dense block
    tf.keras.layers.Dense(256, activation='relu'),   
    tf.keras.layers.Dropout(0.5),  
    tf.keras.layers.Dense(256, activation='relu'),   
    tf.keras.layers.Dropout(0.5),
    
    # Classification layer
    tf.keras.layers.Dense(len(LABELS), activation='softmax')
])
print('MODEL HAS {} PARAMETERS.'.format(model.count_params()))

This is not a huge CNN, it only has ~200,000 parameters. Yet, we also only have a very small dataset with just 27 classes.

Next, we need to specify an optimzer, initial learning rate, a loss function and a metric.

In [ ]:
# Compile the model and specify optimizer, loss and metric
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01),
              metrics=['accuracy'])

Callbacks make our life easier, the three that we're adding will take care of saving the best checkpoint, they will reduce the learning rate whenever the training process stalls, and they will stop the training if the model is overfitting.

In [ ]:
# Add callbacks to reduce the learning rate if needed, early stopping, and checkpoint saving
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  patience=2, 
                                                  verbose=1, 
                                                  factor=0.5),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              verbose=1,
                                              patience=5),
             tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True)]

Here we go, everything is in place, let's train a model. We'll use 20% of our training data for validation and we'll stop after 25 epochs.

In [ ]:
# Let's train the model for a few epochs
H = model.fit(train_specs, 
          train_labels,
          batch_size=32,
          validation_split=0.2,
          callbacks=callbacks,
          epochs=50)

In [ ]:
history = pd.DataFrame(H.history)
plt.figure()
plt.plot(history)

Not too bad, we got into the 60s of our validation accuracy. But remember, we're training on focal recordings and validate on focal recordings. The scores might not tell us much about how well we will perform on soundscape data.

We'll have to check ourselves, luckily, we have some validation soundscapes.

# 6. Soundscape analysis

In this tutorial, we will simply pick a soundscape from the training data, but the overall process can easily be automated and then applied to all soundscape files. And again, we have to load a file with Librosa, extract spectrograms for 5-second chunks, pass each chunk through the model and eventually assign a label to the 5-second audio chunk.

Let's use a soundscape that actually contains some of the species that we trained our model for. The file "28933_SSW_20170408.ogg" seems to contain a lot of Song Sparrow (sonspa) vocalizations, let's try this one then.

In [ ]:
from custom_layers import SimpleSpecLayer

# Load the best checkpoint
model = tf.keras.models.load_model('./best_model.h5', custom_objects={'SimpleSpecLayer': SimpleSpecLayer})


# Pick a soundscape
soundscape_path = '../input/birdclef-2021/train_soundscapes/28933_SSW_20170408.ogg'

# Open it with librosa
sig, rate = librosa.load(soundscape_path, sr=SAMPLE_RATE)

# Store results so that we can analyze them later
data = {'row_id': [], 'prediction': [], 'score': []}

# Split signal into 5-second chunks
# Just like we did before (well, this could actually be a seperate function)
sig_splits = []
for i in range(0, len(sig), int(SIGNAL_LENGTH * SAMPLE_RATE)):
    split = sig[i:i + int(SIGNAL_LENGTH * SAMPLE_RATE)]

    # End of signal?
    if len(split) < int(SIGNAL_LENGTH * SAMPLE_RATE):
        break

    sig_splits.append(split)
    
# Get the spectrograms and run inference on each of them
# This should be the exact same process as we used to
# generate training samples!
seconds, scnt = 0, 0
for chunk in sig_splits:
    
    # Keep track of the end time of each chunk
    seconds += 3
        
#     # Get the spectrogram
#     hop_length = int(SIGNAL_LENGTH * SAMPLE_RATE / (SPEC_SHAPE[1] - 1))
#     mel_spec = librosa.feature.melspectrogram(y=chunk, 
#                                               sr=SAMPLE_RATE, 
#                                               n_fft=1024, 
#                                               hop_length=hop_length, 
#                                               n_mels=SPEC_SHAPE[0], 
#                                               fmin=FMIN, 
#                                               fmax=FMAX)

#     mel_spec = librosa.power_to_db(mel_spec, ref=np.max) 

#     # Normalize to match the value range we used during training.
#     # That's something you should always double check!
#     mel_spec -= mel_spec.min()
#     mel_spec /= mel_spec.max()
    
#     # Add channel axis to 2D array
#     mel_spec = np.expand_dims(mel_spec, -1)

#     # Add new dimension for batch size
#     mel_spec = np.expand_dims(mel_spec, 0)
    
    # Predict
    p = model.predict(chunk)[0]
    
    # Get highest scoring species
    idx = p.argmax()
    species = LABELS[idx]
    score = p[idx]
    
    # Prepare submission entry
    data['row_id'].append(soundscape_path.split(os.sep)[-1].rsplit('_', 1)[0] + 
                          '_' + str(seconds))    
    
    # Decide if it's a "nocall" or a species by applying a threshold
    if score > 0.25:
        data['prediction'].append(species)
        scnt += 1
    else:
        data['prediction'].append('nocall')
        
    # Add the confidence score as well
    data['score'].append(score)
        
print('SOUNSCAPE ANALYSIS DONE. FOUND {} BIRDS.'.format(scnt))

In [ ]:
# Load the best checkpoint
# model = tf.keras.models.load_model('./best_model.h5', custom_objects={'SimpleSpecLayer': SimpleSpecLayer})
model = tf.keras.models.load_model('./best_model.h5')

# Pick a soundscape
soundscape_path = '../input/birdclef-2021/train_soundscapes/28933_SSW_20170408.ogg'

# Open it with librosa
sig, rate = librosa.load(soundscape_path, sr=SAMPLE_RATE)

# Store results so that we can analyze them later
data = {'row_id': [], 'prediction': [], 'score': []}

# Split signal into 5-second chunks
# Just like we did before (well, this could actually be a seperate function)
sig_splits = []
for i in range(0, len(sig), int(SIGNAL_LENGTH * SAMPLE_RATE)):
    split = sig[i:i + int(SIGNAL_LENGTH * SAMPLE_RATE)]

    # End of signal?
    if len(split) < int(SIGNAL_LENGTH * SAMPLE_RATE):
        break

    sig_splits.append(split)
    
# Get the spectrograms and run inference on each of them
# This should be the exact same process as we used to
# generate training samples!
seconds, scnt = 0, 0
for chunk in sig_splits:
    
    # Keep track of the end time of each chunk
    seconds += 3
        
    # Get the spectrogram
    hop_length = int(SIGNAL_LENGTH * SAMPLE_RATE / (SPEC_SHAPE[1] - 1))
    mel_spec = librosa.feature.melspectrogram(y=chunk, 
                                              sr=SAMPLE_RATE, 
                                              n_fft=1024, 
                                              hop_length=hop_length, 
                                              n_mels=SPEC_SHAPE[0], 
                                              fmin=FMIN, 
                                              fmax=FMAX)

    mel_spec = librosa.power_to_db(mel_spec, ref=np.max) 

    # Normalize to match the value range we used during training.
    # That's something you should always double check!
    mel_spec -= mel_spec.min()
    mel_spec /= mel_spec.max()
    
    # Add channel axis to 2D array
    mel_spec = np.expand_dims(mel_spec, -1)

    # Add new dimension for batch size
    mel_spec = np.expand_dims(mel_spec, 0)
    
    # Predict
    p = model.predict(mel_spec)[0]
    
    # Get highest scoring species
    idx = p.argmax()
    species = LABELS[idx]
    score = p[idx]
    
    # Prepare submission entry
    data['row_id'].append(soundscape_path.split(os.sep)[-1].rsplit('_', 1)[0] + 
                          '_' + str(seconds))    
    
    # Decide if it's a "nocall" or a species by applying a threshold
    if score > 0.25:
        data['prediction'].append(species)
        scnt += 1
    else:
        data['prediction'].append('nocall')
        
    # Add the confidence score as well
    data['score'].append(score)
        
print('SOUNSCAPE ANALYSIS DONE. FOUND {} BIRDS.'.format(scnt))

Ok, we found a few bird species with a score above the threshold. Let's look at the results and see how well we're actually doing.

In [ ]:
# Make a new data frame
results = pd.DataFrame(data, columns = ['row_id', 'prediction', 'score'])

# Merge with ground truth so we can inspect
gt = pd.read_csv('../input/birdclef-2021/train_soundscape_labels.csv',)
results = pd.merge(gt, results, on='row_id')

# Let's look at the first 50 entries
results.head(114)

Ok, that's not too bad. We actually got some of these Song Sparrow (sonspa) vocalizations. Well, and we missed others... We also didn't detect the Northern Cardinal (norcar) and Red-winged Blackbird (rewbla) even though we had them in our training data.

This is a good example for the difficulties we're facing when analyzing soundscapes. Focal recordings as training data can be misleading and soundscapes have much higher noise levels (and also contain very faint bird calls).

Now it's your turn to find better strategies to cope with this shift in acoustic domains. Please don't hesitate to leave a comment or start a new forum thread if you have any questions.